In [1]:
# Import needed libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import utilities
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [2]:
# Check Tensorflow version
print('Tensorflow version: ',tf.__version__)
# Check if GPU is being used
print('GPU available: ', tf.config.list_physical_devices('GPU'))
# If a GPU is being used, check that it's not being used by another process as well
if tf.config.list_physical_devices('GPU'):
  print("GPU usage: ", tf.config.experimental.get_memory_info('GPU:0'))

Tensorflow version:  2.10.1
GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU usage:  {'current': 0, 'peak': 0}


## Load the data

In [ ]:
# Training set
x_train, filenames = utilities.load_images('./data/pre_processed/training/raw_resized_250px')
y_train = utilities.get_labels( labels_path='./data/pre_processed/training/train.csv',
                                filenames_list=filenames,
                                key_col='id',
                                value_col='boneage')

In [ ]:
#Reduce the number of samples
# percentage = 0.3 # or any other float value between 0 and 1
# num_samples = int(len(x_train) * percentage)
# x_train = x_train[:num_samples]
# y_train = y_train[:num_samples]

In [ ]:
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_train bytes: ', x_train.nbytes/1000000, 'MB')
print('Type of elements: ',type(x_train[0][0][0][0]))

In [ ]:
# Validation set
x_validate, filenames = utilities.load_images('./data/pre_processed/validation/validation_1/raw_resized_250px')
y_validate = utilities.get_labels( labels_path='./data/pre_processed/validation/validation.csv',
                                filenames_list=filenames,
                                key_col='Image ID',
                                value_col='Bone Age (months)')

In [ ]:
print('x_validate.shape: ', x_validate.shape)
print('y_validate.shape: ', y_validate.shape)

## Preprocess the data

In [ ]:
# Normalize the data
x_train, x_validate = x_train/255, x_validate/255 # Normalize the data
print('x_train.shape: ', x_train.shape)
print('x_train bytes: ', x_train.nbytes/1000000, 'MB')
print('Type of elements: ',type(x_train[0][0][0][0]))

## Instantiate the model

In [ ]:
# Build the model using the functional API
i = Input(shape=x_train[0].shape)
x = Conv2D(8, (3,3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(8, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(16, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(16, (3,3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(32, (3,3), strides=2, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), strides=2, activation='relu', padding='same' )(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)

model = Model(i,x)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
model.summary()

## Training ("fit") the model

In [ ]:
# Fit the model
batch_size = 32  # or any other integer value that works for your system
r = model.fit(x_train, y_train, validation_data=(x_validate, y_validate), epochs=100, batch_size=batch_size)

## Save the model

In [ ]:
# Let's now save our model to a file
model.save('./models/cnn.h5')